In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Performing interactive authentication. Please follow the instructions on the terminal.
Interactive authentication successfully completed.
Workspace name: optimize-proj
Azure region: eastus2
Subscription id: 6b66a03f-9834-4d5b-92f6-b82e733674dc
Resource group: azuremleast


To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code AGWGDN9PF to authenticate.


In [12]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

clusters = [("lightweight", "STANDARD_D2_V2"),
            ("heavyweight", "STANDARD_DS3_V2")]

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

try:
    compute_target = ComputeTarget(workspace=ws, name=clusters[1][0])
    print('Found existing cluster, use it.')
except ComputeTargetException:
    print('Creating a new compute cluster...')
    compute_config = AmlCompute.provisioning_configuration(vm_size=clusters[1][1], max_nodes=4)
    compute_target = ComputeTarget.create(ws, clusters[1][0], compute_config)

compute_target.wait_for_completion(show_output=True)


Creating a new compute cluster...
InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [13]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        '--C' : choice(0.001,0.01,0.1,1),
        '--max_iter': choice(5,10,25,50,100)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory="./",
                    script="train.py",
                    compute_target=compute_target,
                    environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                                    hyperparameter_sampling=ps,
                                    policy=policy,
                                    primary_metric_name="Accuracy",
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=4,
                                    max_concurrent_runs=4
                                    )

In [14]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run = exp.submit(hyperdrive_config)
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [17]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])

Best Run Id:  HD_1fd53c80-5b7a-4663-81d9-410998a80749_2

 Accuracy: 0.9036418816388467


In [16]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(["https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"])

In [9]:
from train import clean_data
import pandas as pd 

# Use the clean_data function to clean your data.
x, y = clean_data(ds)
clean_df = pd.concat([x, y], axis=1)
clean_df.head()

In [22]:
datastore = ws.get_default_datastore()
clean_ds= TabularDatasetFactory.register_pandas_dataframe(dataframe=clean_df,
                                                            target=datastore,
                                                            name="project1-data")

Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/41de444c-a096-4a86-a976-180d6958a661/
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'emp.var.rate' -> 'emp_var_rate'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'cons.price.idx' -> 'cons_price_idx'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'cons.conf.idx' -> 'cons_conf_idx'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'nr.employed' -> 'nr_employed'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'job_admin.' -> 'job_admin_'
Column header contains '.' This period will be translated to '_' as we write the data out to parquet files: 'education_basic.4y'

In [23]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    compute_target=compute_target,
    task="classification",
    primary_metric="accuracy",
    training_data=clean_ds,
    label_column_name="y",
    n_cross_validations=5,
    enable_early_stopping=True,
    blocked_models=["KNN", "LinearSVM"])

In [24]:
# Submit your automl run
automl_exp = Experiment(ws, 'udacity-automl')
automl_run = automl_exp.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-automl,AutoML_49599d3a-9a8d-457c-af4a-7bfc20018a6e,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


In [26]:
automl_run.wait_for_completion(show_output=True)

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-automl,AutoML_49599d3a-9a8d-457c-af4a-7bfc20018a6e,automl,Running,Link to Azure Machine Learning studio,Link to Documentation



Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+--------------------------------------+
|Size of the smallest class    |Name/Label of the smallest class|Number of samples in the training data|
+==============================+================================+======================================+
|3692                          |1                               |32950                                 |
+---------

{'runId': 'AutoML_49599d3a-9a8d-457c-af4a-7bfc20018a6e',
 'target': 'heavyweight',
 'status': 'Completed',
 'startTimeUtc': '2022-09-20T23:38:25.72312Z',
 'endTimeUtc': '2022-09-21T00:19:22.246166Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'Experiment timeout reached, hence experiment stopped. Current experiment timeout: 0 hour(s) 30 minute(s)'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'heavyweight',
  'AMLSettingsJsonString': '{"path":null,"name":"udacity-automl","subscription_id":"6b66a03f-9834-4d5b-92f6-b82e733674dc","resource_group":"azuremleast","workspace_name":"optimize-proj","region":"eastus2","compute_target":"heavyweight","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":true,"enabl

In [29]:
# Retrieve and save your best automl model.

automl_best_run, automl_best_model = automl_run.get_output()

In [34]:
automl_best_run.download_file('outputs/model.pkl', 'automl_best_model.pkl')